In [2]:
#importing libraries
import os
import numpy as np
import cv2
import os
import shutil
import random
import tensorflow as tf
import imgaug.augmenters as iaa
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model 
from tensorflow.keras.optimizers import Adam


2024-05-20 23:02:05.829442: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 23:02:05.878921: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 23:02:06.094503: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 23:02:07.427654: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Setting Dataset Path
images_base_directory = "../samaan hamara/final_images/"
# images_base_directory = "./my_generated_images"
# images_base_directory = "./final_images"
# Function to get and shuffle image files
def get_and_shuffle_image_files(directory):
    image_files = os.listdir(directory)
    random.shuffle(image_files)
    return image_files



In [4]:
# need to get images names because ismein labels hain

def load_data(image_files):
    images = []
    labels = []
    for file in image_files:
        img = load_img(images_base_directory + "/" +  file, color_mode='grayscale')
        img = img_to_array(img) / 255.0
        label = os.path.basename(file).split('.')[0]
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)

def get_image_files(directory):
    return os.listdir(directory)


image_files = get_image_files(images_base_directory)    
image_files = get_and_shuffle_image_files(images_base_directory)


In [5]:
print(image_files)

['893253.png', '357454.png', '798435.png', '385282.png', '642486.png', '485756.png', '252897.png', '748352.png', '108873.png', '107490.png', '494398.png', '959878.png', '113011.png', '307325.png', '875636.png', '763678.png', '385382.png', '984953.png', '161826.png', '586828.png', '238623.png', '965892.png', '108178.png', '583453.png', '106499.png', '104773.png', '368548.png', '487583.png', '657943.png', '575438.png', '579879.png', '796986.png', '859393.png', '976247.png', '782454.png', '756487.png', '392578.png', '104101.png', '856736.png', '852853.png', '264578.png', '105244.png', '452948.png', '115159.png', '263735.png', '108855.png', '116447.png', '111817.png', '784876.png', '106715.png', '109476.png', '369482.png', '949735.png', '787938.png', '675487.png', '674547.png', '695986.png', '475968.png', '689465.png', '848724.png', '982579.png', '984942.png', '754695.png', '764572.png', '463864.png', '243434.png', '853758.png', '362723.png', '245342.png', '268792.png', '607087.png', '3698

In [6]:
display(load_data(image_files[0:5])[0].shape)

(5, 80, 190, 1)

### processing labels and creating categorical label

In [7]:
# only loading 80% of data for training
images, labels = load_data(image_files)

# Preprocess the labels

def preprocess_labels(labels, num_characters, num_classes):
    processed_labels = []
    for label in labels:
        # converting individual to int because kaam aayenge
        label = [int(char) for char in label]
        # Convert each integer to categorical
        categorical_label = np.zeros((num_characters, num_classes))
        for i, char in enumerate(label):
            categorical_label[i, char] = 1
        processed_labels.append(categorical_label)
    return np.array(processed_labels)



num_characters = 6  # no of chars
num_classes = 10  # Digits 0-9
labels = preprocess_labels(labels, num_characters, num_classes)


In [8]:
print(labels.shape)

(5710, 6, 10)


In [9]:


def create_model(input_shape, num_classes, num_characters):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(num_characters * num_classes, activation='softmax'))
    model.add(Reshape((num_characters, num_classes)))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = images[0].shape
model = create_model(input_shape, num_classes, num_characters)

model.fit(images, labels, epochs=16, batch_size=32, validation_split=0.2)
print(input_shape)

/home/panda/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-05-20 23:02:37.809562: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-20 23:02:37.810364: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and

Epoch 1/16


2024-05-20 23:02:38.141191: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 277734400 exceeds 10% of free system memory.


143/143 ━━━━━━━━━━━━━━━━━━━━ 89s 607ms/step - accuracy: 0.1477 - loss: 2.2137 - val_accuracy: 0.2373 - val_loss: 1.9726
Epoch 2/16
143/143 ━━━━━━━━━━━━━━━━━━━━ 86s 603ms/step - accuracy: 0.3169 - loss: 1.7503 - val_accuracy: 0.6559 - val_loss: 0.8859
Epoch 3/16
143/143 ━━━━━━━━━━━━━━━━━━━━ 87s 611ms/step - accuracy: 0.6278 - loss: 0.9648 - val_accuracy: 0.8398 - val_loss: 0.4943
Epoch 4/16
143/143 ━━━━━━━━━━━━━━━━━━━━ 87s 609ms/step - accuracy: 0.7807 - loss: 0.6304 - val_accuracy: 0.8813 - val_loss: 0.3684
Epoch 5/16
143/143 ━━━━━━━━━━━━━━━━━━━━ 88s 615ms/step - accuracy: 0.8385 - loss: 0.4697 - val_accuracy: 0.9095 - val_loss: 0.2866
Epoch 6/16
143/143 ━━━━━━━━━━━━━━━━━━━━ 87s 609ms/step - accuracy: 0.8667 - loss: 0.3914 - val_accuracy: 0.9297 - val_loss: 0.2288
Epoch 7/16
143/143 ━━━━━━━━━━━━━━━━━━━━ 87s 611ms/step - accuracy: 0.8890 - loss: 0.3231 - val_accuracy: 0.9470 - val_loss: 0.1885
Epoch 8/16
143/143 ━━━━━━━━━━━━━━━━━━━━ 88s 618ms/step - accuracy: 0.9077 - loss: 0.2740 - val

In [14]:


# Function to resize and normalize an image
def preprocess_image(image_file, target_size):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)  # Load the image in grayscale
    if img is None:
        raise ValueError(f"Image {image_file} not found.")
    img = cv2.resize(img, target_size) 
    img = img / 255.0  
    # adding a channel dimension to the last 
    img = np.expand_dims(img, axis=-1) 
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to predict the CAPTCHA
def predict_captcha(model, preprocessed_img):
    prediction = model.predict(preprocessed_img)
    print(prediction[0])
    predicted_label = ''.join([str(np.argmax(char)) for char in prediction[0]])
    return predicted_label

# Predict a new CAPTCHA image
image_file = images_base_directory + '/' + "275394.png"
preprocessed_img = preprocess_image(image_file, (190, 80))  # Resize to 190x80
captcha = predict_captcha(model, preprocessed_img)
print(f'Predicted CAPTCHA: {captcha}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[3.49851504e-25 1.26960458e-16 2.72682490e-04 2.07931922e-10
  2.40967624e-09 1.51527870e-11 1.73699485e-10 3.55013588e-12
  5.06522491e-10 1.66828662e-10]
 [2.82289248e-17 3.41813925e-16 4.21863890e-21 3.86636972e-10
  2.06575645e-10 1.55582700e-10 2.05243662e-08 2.52900715e-03
  3.09581215e-11 7.58394405e-14]
 [2.29242400e-16 6.81039239e-17 3.01060488e-09 3.64950181e-09
  2.26943866e-10 9.51331691e-04 1.62843211e-10 8.67914143e-11
  2.22579644e-08 2.10409157e-09]
 [1.61775363e-15 1.40500796e-16 4.97247044e-09 4.03830782e-02
  5.58035944e-11 6.17658280e-09 6.38993788e-11 3.01548582e-08
  4.34193903e-10 3.73313908e-10]
 [2.26386238e-13 3.78199322e-15 5.40788291e-12 2.56323829e-09
  1.72197034e-09 1.49182871e-08 3.24706539e-10 1.14363102e-11
  8.23336288e-10 1.19349115e-01]
 [2.72422991e-14 9.53880436e-16 1.35097644e-09 1.14985216e-10
  8.36514711e-01 7.05390824e-12 9.26889902e-12 3.81836659e-14
  1.90926275e-10 6.63534019e-11]]
Predicted CAPTCHA: 

In [15]:
model.save("model_trained_on_5710_images.h5")